In [2]:
import faculty.datasets as datasets
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import spacy
%matplotlib inline

2023-02-28 15:58:40.202374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda/envs/Python3/lib:
2023-02-28 15:58:40.202431: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-28 15:58:40.202465: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cube-b2f6b542-b29a-4672-9ac9-c005bf019a65-844657fdc8-8rszg): /proc/driver/nvidia/version does not exist


In [4]:
df = pd.read_excel('/project/my-society/data/raw/plans.xlsx')
drop_cols = ['unfound','credit','date_retrieved','time_period','scope','status',
            'well_presented', 'baseline_analysis', 'notes',
            'authority_code', 'authority_type', 'wdtk_id',
           'mapit_area_code', 'country', 'gss_code', 'county', 'region',
           'population', 'unfound','credit','homepage_mention', 'dedicated_page',
            'plan_due','title','title_checked','twitter_url','twitter_name']
# df = df.drop(drop_cols,axis=1)
df.columns

Index(['council', 'search_link', 'unfound', 'credit', 'url', 'date_retrieved',
       'time_period', 'type', 'scope', 'status', 'homepage_mention',
       'dedicated_page', 'well_presented', 'baseline_analysis', 'notes',
       'plan_due', 'title', 'title_checked', 'plan_path', 'file_type',
       'charset', 'authority_code', 'authority_type', 'wdtk_id',
       'mapit_area_code', 'country', 'gss_code', 'county', 'region',
       'population', 'website_url', 'twitter_url', 'twitter_name', 'text'],
      dtype='object')

In [5]:
df.loc[(df['file_type'] == 'pdf') & (df['text'].isna())] #373 NE derbyshire district is pdf but no text pulled


,council,search_link,unfound,credit,url,date_retrieved,time_period,type,scope,status,...,mapit_area_code,country,gss_code,county,region,population,website_url,twitter_url,twitter_name,text
373,North East Derbyshire District Council,https://www.google.com/search?q=North+East+Der...,NaN,CMa,https://www.dropbox.com/s/m18ubpxzm5uwxof/Nort...,01/02/2020,2019-2030,Action plan,NaN,Approved,...,DIS,England,E07000038,DBY,East Midlands,102216,https://www.ne-derbyshire.gov.uk/,NaN,NaN,NaN


In [136]:
df[['file_type','council','text']].groupby('file_type',dropna=False).count()

,council,text
file_type,,
docx,5,0
html,124,17
pdf,503,502
xlsm,1,0
xlsx,2,0
NaN,51,0


Loading in html files

In [69]:
df.loc[df['file_type'].isnull()].describe()

,council,search_link,url,type,plan_path,file_type,charset,website_url,text
count,51,50,0,8,0,0,0,51,0
unique,51,50,0,4,0,0,0,51,0
top,Aberdeenshire Council,https://www.google.com/search?q=Aberdeenshire+...,NaN,Action plan,NaN,NaN,NaN,https://www.aberdeenshire.gov.uk,NaN
freq,1,1,NaN,5,NaN,NaN,NaN,1,NaN


In [97]:
urls = []
for idx in df.loc[df['file_type'] == 'html'].index:
    urls.append(df.loc[df['file_type'] == 'html']['url'][idx])

In [112]:
# !pip install trafilatura

In [101]:
import requests
import trafilatura
text_content = trafilatura.extract(requests.get(urls[0]).text)

In [ ]:
for idx in df.loc[(df['file_type'] == 'html') & (df['text'].isnull())].index:
#     try:
    print('url at index ', idx, 'good')
    url = df['url'][idx]
#     text = trafilatura.extract(requests.get(url).text)
    print(url)
#     print(text)
#     df.loc[idx, 'text'] = text
#     except:
#         print()

In [ ]:
text = trafilatura.extract(requests.get().text)

In [129]:
df.loc[(df['file_type'] == 'html') & (df['text'].isnull())].index

Int64Index([121, 126, 128, 147, 149, 150, 152, 162, 183, 202,
            ...
            649, 651, 655, 658, 663, 669, 672, 676, 678, 679],
           dtype='int64', length=107)

In [130]:
df[['file_type','council','text']].groupby('file_type',dropna=False).count() #have added 17 plans 

,council,text
file_type,,
docx,5,0
html,124,17
pdf,503,502
xlsm,1,0
xlsx,2,0
NaN,51,0


In [141]:
# print(df.loc[(df['file_type'] == 'html')]['text'][12])